# Data Preperations

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!pip install geemap

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TzKEdUkNF3s33TN9gy7ngpVuh_s8Vw0ceGOAi4H3QlQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6IO4fukIBnqqphGaYZ8wC82ylzaRWHN3Vo5XY501mjTc_Anz4YI54

Successfully saved authorization token.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import geemap
import folium
import pickle
import pprint
import json

In [ ]:
# Iowa County codes
def get_counties_dict():
    return {
        19001 : 'Adair',
        19003 : 'Adams',
        19005 : 'Allamakee',
        19007 : 'Appanoose',
        19009 : 'Audubon',
        19011 : 'Benton',
        19013 : 'Black Hawk',
        19015 : 'Boone',
        19017 : 'Bremer',
        19019 : 'Buchanan',
        19021 : 'Buena Vista',
        19023 : 'Butler',
        19025 : 'Calhoun',
        19027 : 'Carroll',
        19029 : 'Cass',
        19031 : 'Cedar',
        19033 : 'Cerro Gordo',
        19035 : 'Cherokee',
        19037 : 'Chickasaw',
        19039 : 'Clarke',
        19041 : 'Clay',
        19043 : 'Clayton',
        19045 : 'Clinton',
        19047 : 'Crawford',
        19049 : 'Dallas',
        19051 : 'Davis',
        19053 : 'Decatur',
        19055 : 'Delaware',
        19057 : 'Des Moines',
        19059 : 'Dickinson',
        19061 : 'Dubuque',
        19063 : 'Emmet',
        19065 : 'Fayette',
        19067 : 'Floyd',
        19069 : 'Franklin',
        19071 : 'Fremont',
        19073 : 'Greene',
        19075 : 'Grundy',
        19077 : 'Guthrie',
        19079 : 'Hamilton',
        19081 : 'Hancock',
        19083 : 'Hardin',
        19085 : 'Harrison',
        19087 : 'Henry',
        19089 : 'Howard',
        19091 : 'Humboldt',
        19093 : 'Ida',
        19095 : 'Iowa',
        19097 : 'Jackson',
        19099 : 'Jasper',
        19101 : 'Jefferson',
        19103 : 'Johnson',
        19105 : 'Jones',
        19107 : 'Keokuk',
        19109 : 'Kossuth',
        19111 : 'Lee',
        19113 : 'Linn',
        19115 : 'Louisa',
        19117 : 'Lucas',
        19119 : 'Lyon',
        19121 : 'Madison',
        19123 : 'Mahaska',
        19125 : 'Marion',
        19127 : 'Marshall',
        19129 : 'Mills',
        19131 : 'Mitchell',
        19133 : 'Monona',
        19135 : 'Monroe',
        19137 : 'Montgomery',
        19139 : 'Muscatine',
        19141 : 'O Brien',
        19143 : 'Osceola',
        19145 : 'Page',
        19147 : 'Palo Alto',
        19149 : 'Plymouth',
        19151 : 'Pocahontas',
        19153 : 'Polk',
        19155 : 'Pottawattamie',
        19157 : 'Poweshiek',
        19159 : 'Ringgold',
        19161 : 'Sac',
        19163 : 'Scott',
        19165 : 'Shelby',
        19167 : 'Sioux',
        19169 : 'Story',
        19171 : 'Tama',
        19173 : 'Taylor',
        19175 : 'Union',
        19177 : 'Van Buren',
        19179 : 'Wapello',
        19181 : 'Warren',
        19183 : 'Washington',
        19185 : 'Wayne',
        19187 : 'Webster',
        19189 : 'Winnebago',
        19191 : 'Winneshiek',
        19193 : 'Woodbury',
        19195 : 'Worth',
        19197 : 'Wright',
    }

## Getting the Image Collections

In [ ]:
# Crop Season
start_date = '2019-04-01'
end_date = '2019-11-01'
area = 'iowa_simplified.geojson'


collections = [
    'NASA/ORNL/DAYMET_V4',
    'USDA/NASS/CDL'
]

# Same as counties dataset
out_crs = 'EPSG:32145'

In [ ]:
def make_aoi(shape_file):
    with open (shape_file, 'r') as f:
        borders = json.load(f)

    iowa_geometry = borders['features'][0]['geometry']
    pprint(iowa_geometry)
    return ee.Geometry(iowa_geometry)

In [ ]:
def get_collections(aoi, collections, start_date, end_date):
    date_filter = ee.Filter.date(start_date, end_date)
    image_collections = []
    for collection in collections:
        image_collections.append(ee.ImageCollection(collection).filterBounds(aoi).filterDate(start_date, end_date))

    # inner_join = ee.Join.inner()
    return image_collections
    
aoi = ee.FeatureCollection('TIGER/2018/States').filterMetadata('NAME', 'equals', 'Iowa').geometry()

### Grabbing the image collections and masking crops

In [ ]:
image_collections = get_collections(aoi, collections[:-1], start_date, end_date)
crop_collection = ee.ImageCollection(collections[-1]).filterBounds(aoi).filterDate('2018-01-01', '2019-12-31')

crop_collection = crop_collection.map(lambda image: image.clip(aoi))

# is cropland
crops = crop_collection.select(['confidence']).mean()
crop_threshold = 95
ee_crop_threshold = ee.Image.constant(crop_threshold)
is_crop_mask = crops.gte(ee_crop_threshold)

# is corn
crop_types = crop_collection.select(['cropland']).mode()
corn = 1
ee_corn = ee.Image.constant(corn)
is_corn_mask = crop_types.eq(ee_corn)

# is cultivated
cultivated = crop_collection.select(['cultivated']).mean()
cultivated_threshold = 1.5
ee_cultivated_threshold = ee.Image.constant(cultivated_threshold)


final_mask = cultivated.gte(ee_cultivated_threshold).bitwise_and(is_crop_mask).bitwise_and(is_corn_mask)
final_mask = final_mask.clip(aoi)

# inverse of the final mask
isnt_crop_mask = crops.lt(ee_crop_threshold)
inv_mask = cultivated.lt(cultivated_threshold).bitwise_and(isnt_crop_mask)
inv_mask = inv_mask.clip(aoi)

def pseudomask_image(image, final_mask=final_mask, inv_mask=inv_mask):
  mask = image.updateMask(final_mask)
  low = ee.Image.constant(-99)
  masked = low.blend(mask)
  return masked 


# reproject to the same crs and resolution 
for i in range(len(image_collections)):
    image_collections[i] = image_collections[i].map(lambda image: image.updateMask(final_mask))
    image_collections[i] = image_collections[i].map(lambda image: image.clip(aoi))
    # image_collections[i] = image_collections[i].map(lambda img: img.reproject(out_crs, scale=3500))

### Making Histograms:




The idea here is to iterate over the counties, and for each one to get a histogram for all of the bands in the image collection. The histogram is over both space and time (where space refers to only the crop land). 


This is where we run into problems

In [ ]:
def compute_histograms(image_collections, steps=100, filename='/content/drive/MyDrive/RemoteSensing/RS_FinalProj/'):
  counties_dict = get_counties_dict()
  min_reducer = ee.Reducer.min()
  max_reducer = ee.Reducer.max()
  for fip_code, name in counties_dict.items():
    hists = {}
    print(f'{name} County:')
    for i, ic in enumerate(image_collections):
      print(f'Image Collection Index: {i}')
      county_aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', f'{(fip_code-19000):03}')
      # state-wide maxs and mins temporally
      state_mins, state_maxs = (ic.min(), ic.max())

      # reduce the max and mins spatially
      band_mins = county_mins.reduceRegion(reducer=min_reducer, geometry=county_aoi, scale=3500)
      band_maxs = county_maxs.reduceRegion(reducer=max_reducer, geometry=county_aoi, scale=3500)

      # make into floats
      state_mins, state_maxs = (band_mins.getInfo(), band_maxs.getInfo())
      county_ic = ic.map(lambda image : image.clip(county_aoi))
      
      # join the mins and maxs into tuples
      band_stats = {}
      for k, v in state_mins.items():
        band_stats[k] = (v, state_maxs[k])
      print('Extracted image collection mins and maxs:')
      print(band_stats)

      # extract band by band
      for band_name, (band_min, band_max) in band_stats.items():
        if (band_min is not None) and (band_max is not None):
          # get the band we want              
          county_ic_forBand = county_ic.select(band_name)
          # create a band-specific fixed-histogram reducer with the specified bin number
          hist_reducer = ee.Reducer.fixedHistogram(min=band_min, 
                                                    max=band_max+1,   #max is not inclusive
                                                    steps=steps)
          # map image collection into a feature collection with all of the image histograms
          feature_hists = county_ic_forBand.map(lambda img: ee.Feature(None, {'hist': img.reduceRegion(hist_reducer).get(band_name)}))
          # stack all of the histograms into one big array
          histograms_array = np.array(feature_hists.aggregate_array('hist').getInfo())

          # if we get an output
          # extract the count and sum across the temporal dimension, save into a dict
          print(histograms_array.shape)
          if len(histograms_array.shape) == 3:
            hists[band_name] = np.sum(histograms_array[:,:, 1], axis=0)
            print(f'Histogram for {band_name} added')
        
    # save to file
    print('Saving...')
    with open(filename+str(name)+'.p', 'wb') as fout:
      pickle.dump(hists, fout)
  return 

out_dir = '/content/drive/MyDrive/RemoteSensing/RS_FinalProj/'
data = compute_histograms(image_collections[:], 100, filename=out_dir+str(2019)+'/')

## Visualization

In [ ]:
county_aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', f'{(1):03}')

In [ ]:
countyOutlines = ee.Image().float().paint(featureCollection=ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', '025'),
                                          color='black', width=2)

In [ ]:
map1 = geemap.Map(center=(41.7155, -93.0190), zoom=7)

'''
for ic in image_collections:
  bands = ic.first().bandNames().getInfo()
  for band in bands:
    viz = {'bands': [band]}
    map1.addLayer(ic.mosaic(), viz, band )
    break
  break
'''
#hists = {'hist': hists}
calhoun_aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', '025')
map1.addLayer(image_collections[0].mosaic().clip(calhoun_aoi), {}, 'Daymet Mosaic Masked')
map1.addLayer(countyOutlines)
'''
for name, map in hists.items():
  map1.addLayer(map, {}, name)
'''
#map1.addLayer(mn, {}, 'mean')

#map1.addLayer(test, crop_viz, 'Cropland')
map1.addLayer(final_mask.clip(calhoun_aoi), {}, 'Mask')
#map1.addLayer(inv_mask, {}, 'Inv_Mask')

map1

In [ ]:
dummy = geemap.ee_to_numpy(hists['vp'], region=aoi)

In [ ]:
dummy

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# # Define the visualization parameters.
# image_viz_params = {
#     'bands': ['vp'],
#     'min': 0,
#     'max': 2000,
# }

# # Define a map centered on San Francisco Bay.
# map_l8 = folium.Map(location=[41.7155, -93.0190], zoom_start=7)

# # Add the image layer to the map and display it.
# map_l8.add_ee_layer(ic.first(), image_viz_params, 'false color composite')
# display(map_l8)

In [ ]:
    #pprint(image_collections[i].first().getInfo())
    # print('!!!!!!')

    




    
# """

# # DAYMET

# 'system:index': '20190401',
# 'system:time_end': 1554163200000,
# 'system:time_start': 1554076800000

# # DAILY

# 'system:index': '20190401',
# 'system:time_end': 1554163200000,
# 'system:time_start': 1554076800000,

# # T3H
# 'system:index': 'A20190401_0000',
# 'system:time_end': 1554087600000,
# 'system:time_start': 1554076800000
# """